In [1]:
!pip install huggingface_hub
!pip install chromadb
!pip install langchain
!pip install pypdf
!pip install sentence-transformers

^C
ERROR: Operation cancelled by user
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 4.0 MB/s eta 0:00:00 0:00:01


In [88]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import huggingface_hub
from langchain.vectorstores import Chroma
from langchain.chains import conversational_retrieval

In [89]:
# Load the pdf file and split it into smaller chunks
loader = PyPDFLoader("./pdf_doc/Compromis n°1 de vente d'immeuble entier BOUILLAGUET _ CHEZE [36442].pdf")
documents = loader.load()

# Split the documents into smaller chunks
text_splitter = CharacterTextSplitter(
    chunk_size=1000, chunk_overlap=0
)  # Le paramètre chunk_overlap détermine le nombre de caractères qui se chevauchent entre deux segments de texte consécutifs lors du découpage du texte en morceaux (chunks).
texts = text_splitter.split_documents(documents)

In [90]:
# We will use HuggingFace embeddings
embeddings = HuggingFaceEmbeddings()

/home/jbcheze/.cache/pypoetry/virtualenvs/ov-project-pLFMpO-6-py3.11/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [91]:
# Using Chroma vector database to store and retrieve embeddings of our text
db = Chroma.from_documents(texts, embeddings) # Creation de la base de donné vectorielle chroma 
retriever = db.as_retriever(search_kwargs={"k": 2}) # retriever sert à faire des recherches sémantiques.

In [92]:
import os
from dotenv import load_dotenv

In [93]:
# We are using Mistral-7B for this question answering
from langchain_community.llms import VLLMOpenAI
load_dotenv()
mixtral_api_key = os.getenv("MIXTRAL_API_KEY")
MIXTRAL_API_BASE = os.getenv("MIXTRAL_API_BASE")
huggingfacehub_api_token = os.getenv("huggingfacehub_api_token")

llm = VLLMOpenAI(
        openai_api_key=mixtral_api_key,
        openai_api_base=MIXTRAL_API_BASE,
        model_name="mistralai/Mixtral-8X7B-Instruct-v0.1",
        temperature=0.0,
        max_tokens=7000,
    )

In [94]:
from langchain.chains import ConversationalRetrievalChain

# Create the Conversational Retrieval Chain
qa_chain = ConversationalRetrievalChain.from_llm(
    llm, retriever, return_source_documents=True
)

In [95]:
# We will run an infinite loop to ask questions to LLM and retrieve answers untill the user wants to quit
import sys

chat_history = []
while True:
    query = input("Prompt: ")
    # To exit: use 'exit', 'quit', 'q', or Ctrl-D.",
    if query.lower() in ["exit", "quit", "q"]:
        print("Exiting")
        sys.exit()
    result = qa_chain({"question": query, "chat_history": chat_history})
    print("Answer: " + result["answer"] + "\n")
    chat_history.append((query, result["answer"]))

Answer:  The text does not provide information about when Olivier was born.

Answer:   The text does not provide information about Cheze's birthdate.

Answer:  The buyer is not explicitly named in the provided text.

Answer:  The provided document is a set of guidelines for writing a thesis or report. It includes instructions for formatting the document, creating an introduction and conclusion, writing the main body, and including annexes. The document also provides guidelines for the oral presentation of the work, which should last 30 minutes and include a 10-minute PowerPoint presentation followed by 20 minutes of questions and answers. The document emphasizes the importance of proper citation and warns against plagiarism. It also provides specific instructions for formatting the document in Word and creating a PDF version. The document is intended to help students write and present their theses or reports in a clear and professional manner.

Exiting


SystemExit: 

/home/jbcheze/.cache/pypoetry/virtualenvs/ov-project-pLFMpO-6-py3.11/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
